In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
dataset_path = "/content/gdrive/My Drive/datasets/tas-nlp/"

In [4]:
import re

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

print('Keras       ', keras.__version__)
print('TensorFlow  ', tf.__version__)

Keras        2.2.4-tf
TensorFlow   1.13.1


Using TensorFlow backend.


In [5]:

from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

['/device:GPU:0']

In [0]:
# drive.mount("/content/gdrive", force_remount=True)

In [7]:
! ls -la /content/gdrive/'My Drive'/datasets/tas-nlp/prolexitim-merged-1.3.csv
      

-rw------- 1 root root 209682 May 13 09:39 '/content/gdrive/My Drive/datasets/tas-nlp/prolexitim-merged-1.3.csv'


In [0]:
tasnlp_df = pd.read_csv(dataset_path + "prolexitim-merged-1.3.csv", header=0, delimiter="\t")

In [0]:
# tasnlp_df.head()

In [0]:
# We only need text and label
data = tasnlp_df[['Text','alex-a']]

In [38]:
data.head()

,Text,alex-a
0,Era un niño pensando en el granero pensando a ...,NoAlex
1,"Una madre que está consolando a su hijo, despu...",NoAlex
2,Un pantanal con una barca abandonada. A ver qu...,NoAlex
3,"Era un paraje muy bonito, con una barca, un po...",Alex
4,"Era una vez un matrimonio, que se quería muchí...",Alex


In [39]:
print(data.dtypes)
print(data.count())

Text      object
alex-a    object
dtype: object
Text      334
alex-a    318
dtype: int64


In [0]:
data = data.dropna()

In [41]:
data.count()

Text      318
alex-a    318
dtype: int64

In [42]:
data.groupby('alex-a').count()

,Text
alex-a,
Alex,31
NoAlex,242
PosAlex,45


In [0]:
# Join alexithymia and possible alexithymia as one positive class
data['alex-a'] = data['alex-a'].apply(lambda x: x.replace('PosAlex', 'Alex'))

In [44]:
data.groupby('alex-a').count()

,Text
alex-a,
Alex,76
NoAlex,242


In [0]:
# Cleanining text

import unicodedata
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])
                    

In [0]:
# Cleanining text

data['Text'] = data['Text'].apply(lambda x: remove_accents(x))
data['Text'] = data['Text'].apply(lambda x: x.lower())
data['Text'] = data['Text'].apply((lambda x: re.sub('[^ña-zA-z0-9\s]', '', x)))

In [51]:
data.tail(n=4)

,Text,alex-a
330,el nino estaba sangrando por la boca porque se...,Alex
331,se habian pasado todo el dia corriendo y jugan...,Alex
332,hay unos toros que estan enfadados y se prepar...,Alex
333,un senor que se arrepiente de haber matado a s...,Alex


In [52]:
nb_alex = data[data['alex-a'] == 'Alex'].size
nb_no_alex = data[data['alex-a'] == 'NoAlex'].size


print(f"There are {nb_alex} positive intances of alexithymia or possible alexithymia.")
print(f"There are {nb_no_alex} negative instances, no alexithymia detected")


There are 152 positive intances of alexithymia or possible alexithymia.
There are 484 negative instances, no alexithymia detected


In [0]:
# Keras vectorization of the text corpus 
max_features = 2000

tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['Text'].values)

X = tokenizer.texts_to_sequences(data['Text'].values)

In [61]:
print(f"Text: {data['Text'][1]}. Vector: {X[1]}") 
print(f"Text: {data['Text'][2]}. Vector: {X[2]}") 

Text: una madre que esta consolando a su hijo despues de darle las notas que ha sacado no se cuantos insuficientes. Vector: [8, 160, 1, 16, 870, 5, 11, 125, 45, 2, 871, 18, 397, 1, 38, 872, 12, 10, 539, 873]
Text: un pantanal con una barca abandonada a ver que estan haciendo los de la barca buscar setas. Vector: [4, 874, 14, 8, 161, 540, 5, 112, 1, 39, 126, 19, 2, 7, 161, 212, 875]


In [0]:
# Make sure we have the same length in all document vector
X = pad_sequences(X)

In [74]:
print(f"Vector Length: {len(X[1])}")
print(X[1])
print(f"Vector Length: {len(X[131])}")
print(X[131])
print(f"Vector Length: {len(X[3])}")
print(X[3])

Vector Length: 162
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   8 160
   1  16 870   5  11 125  45   2 871  18 397   1  38 872  12  10 539 873]
Vector Length: 162
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0

In [0]:
# One HOT encoding for alexithymia label
# and the values in Y. 

Y = pd.get_dummies(data['alex-a']).values

In [112]:
# Train / Test partitions (90/10)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)

print(f"Train shape. Feature Vector:{X_train.shape}, Label:{Y_train.shape} (binary classification)")
print(f"Test shape. Feature Vector: {X_test.shape}, Label:{Y_test.shape} (binary classification)")

Train shape. Feature Vector:(286, 162), Label:(286, 2) (binary classification)
Test shape. Feature Vector: (32, 162), Label:(32, 2) (binary classification)


In [96]:
# Deep RNN Model Definition:
# Embedding -> Dropout -> LSTM -> Dense 

embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 162, 128)          256000    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 162, 128)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [85]:
# Train the model
batch_size = 32
model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, verbose = 2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/15
 - 4s - loss: 0.6610 - acc: 0.7098
Epoch 2/15
 - 2s - loss: 0.5484 - acc: 0.7622
Epoch 3/15
 - 2s - loss: 0.5321 - acc: 0.7622
Epoch 4/15
 - 2s - loss: 0.5023 - acc: 0.7622
Epoch 5/15
 - 2s - loss: 0.4435 - acc: 0.7657
Epoch 6/15
 - 2s - loss: 0.3706 - acc: 0.7867
Epoch 7/15
 - 2s - loss: 0.2553 - acc: 0.9021
Epoch 8/15
 - 2s - loss: 0.1375 - acc: 0.9580
Epoch 9/15
 - 2s - loss: 0.1077 - acc: 0.9510
Epoch 10/15
 - 2s - loss: 0.0671 - acc: 0.9895
Epoch 11/15
 - 2s - loss: 0.0324 - acc: 1.0000
Epoch 12/15
 - 2s - loss: 0.0265 - acc: 0.9930
Epoch 13/15
 - 2s - loss: 0.0264 - acc: 0.9930
Epoch 14/15
 - 2s - loss: 0.0172 - acc: 1.0000
Epoch 15/15
 - 2s - loss: 0.0100 - acc: 0.9965


In [108]:
print(X_test.shape)
print(Y_test.shape)
print(f"Test shape. Feature Vector: {X_test.shape}, Label:{Y_test.shape} (binary classification)")

(32, 162)
(32, 2)
Test shape. Feature Vector: (32, 162), Label:(32, 2) (binary classification)


In [113]:
validation_size = 20

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]

score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)

print(model.metrics_names)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

['loss', 'acc']
score: 0.69
acc: 0.50


In [157]:
narratives = ['Este niño está triste porque no le dejan tocar la guitarra, que es lo que le gusta a él. Y se siente mal por tener que aprender a tocar el violín.', 
             'Me pongo muy nerviosos viendo esta imagen. Es una locura']
p_narratives =[]

for text in narratives:
  p_narratives.append(remove_accents(text).lower())
  

print(p_narratives)

['este nino esta triste porque no le dejan tocar la guitarra, que es lo que le gusta a el. y se siente mal por tener que aprender a tocar el violin.', 'me pongo muy nerviosos viendo esta imagen. es una locura']


In [158]:
# vectorizing the text by the pre-fitted tokenizer instance

pp_narratives = tokenizer.texts_to_sequences(p_narratives)

print(pp_narratives)

[[97, 22, 16, 68, 28, 12, 13, 40, 7, 1119, 1, 27, 20, 1, 13, 178, 5, 6, 3, 10, 190, 169, 15, 120, 1, 93, 5, 40, 6, 21], [54, 36, 257, 16, 287, 27, 8]]


In [159]:
# Padding the tweet to have exactly the same shape as `embedding_2` input

pp_narratives = pad_sequences(pp_narratives, maxlen=162, dtype='int32', value=0)
print(pp_narratives)


[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   97   22   16   68   28   12   13   40
     7 1119    1   27   20    1   13  178    5    6    3   10  190  169
    15  120    1   93    5   40    6   21]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0

In [160]:
# Testing the predictive model

alex_screening = model.predict(pp_narratives, batch_size=1, verbose = 2)

for prediction in alex_screening:
  print(prediction)
  if(np.argmax(prediction) == 0):
      print("negative: No Alexithymia")
  elif (np.argmax(prediction) == 1):
    print("positive: Possible Alexithymia")

[0.5027526 0.4972474]
negative: No Alexithymia
[0.49610397 0.503896  ]
positive: Possible Alexithymia
